# Multi-column Indexing

We'll work with factbook.db, a SQLite database that contains information about each country in the world. We'll be working with the facts table in the database. Each row in facts represents a single country, and contains several columns, including: 
- `name` -- the name of the country. 
- `area` -- the total land and sea area of the country.
- `population` -- the population of the country.
- `birth_rate` -- the birth rate of the country.
- `created_at` -- the date the record was created.
- `updated_at` -- the date the record was updated.

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("factbook.db")

In [4]:
query_plan_one = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
query_plan_one

[(0, 0, 0, 'SCAN TABLE facts')]

In [5]:
conn.execute("CREATE INDEX pop_idx ON facts(population)")

In [6]:
conn.execute("CREATE INDEX pop_growth_idx ON facts(population_growth)")

In [7]:
query_plan_two  = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
query_plan_two

[(0,
  0,
  0,
  'SEARCH TABLE facts USING INDEX pop_growth_idx (population_growth<?)')]

In [8]:
conn.execute("CREATE INDEX pop_pop_growth_idx ON facts(population,population_growth)")

In [9]:
query_plan_three = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
query_plan_three

[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_pop_growth_idx (population>?)')]

In [10]:
conn.execute("CREATE INDEX IF NOT EXISTS pop_pop_growth_idx ON facts(population,population_growth)")

In [11]:
query_plan_four = conn.execute("EXPLAIN QUERY PLAN SELECT population, population_growth FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
query_plan_four

[(0,
  0,
  0,
  'SEARCH TABLE facts USING COVERING INDEX pop_pop_growth_idx (population>?)')]

In [ ]:
conn.execute("CREATE INDEX IF NOT EXISTS pop_pop_growth_idx ON facts(population,population_growth)")

In [12]:
query_plan_five = conn.execute("EXPLAIN QUERY PLAN SELECT population FROM facts WHERE population > 1000000;").fetchall()
query_plan_five

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_idx (population>?)')]